In [7]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [3]:
with open('all_children.json', 'r', encoding='utf-8') as file:
    data = json.load(file)


In [ ]:
def getChildInfo(number):
    # URL сторінки
    url = f"https://www.msp.gov.ua/children/child.php?n={number}"

    # Виконання HTTP-запиту
    response = requests.get(url)

    # Перевірка статусу відповіді
    if response.status_code == 200:
        # get object soup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        name_age = soup.find(class_='passport__title-name').text #get name & age
        name = name_age.split(',')[0] #get name
        age = (name_age.split(' ')[1]) #get age
        child_id = soup.find(class_='passport__title-number').text.split('№')[1] #get id

        # get block passport__content with information with child
        passport_content = soup.find('div', class_='row passport__content')
        gender_element = passport_content.find('span', class_='passport__content-title', string="Стать:")
        if gender_element:
            gender = gender_element.find_next('span', class_='passport__content-inner').text.strip()
        else:
            print("Стать не знайдено.")
        
        # get region
        region_element = passport_content.find('span', class_='passport__content-title', string="Область:")
        if region_element:
            region = region_element.find_next('span', class_='passport__content-inner').text.strip()
        else:
            print("Область не знайдено.")
        
        #get disablement status
        disablement_element = passport_content.find('span', class_='passport__content-title', string="Наявність інвалідності:")
        if disablement_element:
            disablement = disablement_element.find_next('span', class_='passport__content-inner').text.strip()
            if disablement == 'є':
                disablement = True
            else:
                disablement = False

        else:
            print("Статус інвалідності не знайдено")

        #get form of adoption
        adoptionForm_element = passport_content.find('span', class_='passport__content-title', string="Можлива форма влаштування:")
        if adoptionForm_element:
            adoptionForm = adoptionForm_element.find_next('span', class_='passport__content-inner').text.strip()
        else:
            print("Форому всиновлення не знайдено.")

        #place where child is now
        place_element = passport_content.find('span', class_='passport__content-title', string="Куди звертатись:")
        if place_element:
            place = place_element.find_next('span', class_='passport__content-inner').text.strip()
        else:
            print("Місце перебування не знайдено.")

        #photo
        photo_element = passport_content.find('a', class_='js-lightBoxGallery')
        if photo_element:
            photo = 'https://www.msp.gov.ua/'+photo_element.find('img').get('src')
        else:
            print("Фотокартка не знайдена.")
        
        #get list of siblings, their names and links to their pages


        # #siblings
        # siblings_element = passport_content.find('ul', class_='passport__content-inner passport__content-inner--brother')
        # if siblings_element:
        #     siblings_count = len(siblings_element.findAll('li'))            
        #     siblings_list = siblings_element.findAll('a')
        #     print(siblings_list[0])

        # else:
        #     print("Братів не знайдено")

        print('name:',name, '\nage:',age, '\nid:',child_id, '\ngender:', gender, '\nregion: ',region,
               '\ndisablement: ', disablement, "\nadoptionForm: ", adoptionForm, "\nplace: ", place, "\nphoto: ", photo,
              '\n--------------------------------------------------')
        # test = pd.DataFrame({'name': [name], 'age': [age], 'siblings': [2]})
        # newRow = pd.DataFrame({'age':[10], 'name': ['Vasyl']})
        # answer = pd.concat([test, newRow], ignore_index=True)
        
        # print(answer)
        
    
    else:
        print(f"Не вдалося отримати сторінку, статус-код: {response.status_code}")

#перевірка функціоналу
for i in range(1,50):
     getChildInfo(data[i])
    # getChildInfo('388104')

name: Валентин 
age: 16 
id: 6829 
gender: чоловіча 
region:  Вінницька обл. 
disablement:  False 
adoptionForm:  національне усиновлення; 
place:  ССД Жмеринської райдержадміністрації: 04332  5-00-50 
photo:  https://www.msp.gov.ua/../files/children/02_children18.01.2025.zip/36367557.jpeg 
--------------------------------------------------
name: Ірина 
age: 15 
id: 6831 
gender: жіноча 
region:  Вінницька обл. 
disablement:  False 
adoptionForm:  національне усиновлення; опіка/піклування; прийомна сім'я та дитячий будинок сімейного типу; міждержавне усиновлення; 
place:  ССД Жмеринської райдержадміністрації: 04332  5-00-50; Національна соціальна сервісна служба України Департамент з питань захисту прав дітей та забезпечення стандартів рівності відділ з питань усиновлення та захисту прав дітей: 289-55-39 м. Київ вул. Еспла 
photo:  https://www.msp.gov.ua/../files/children/02_children18.01.2025.zip/36367633.jpeg 
--------------------------------------------------
name: Артем 
age: 17 
i